In [1]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
conn = sqlite3.connect("sales_agent_co_pilot.db")
cursor = conn.cursor()

In [3]:
sales = pd.read_sql("SELECT * FROM sales", conn)
visits = pd.read_sql("SELECT * FROM visits", conn)
visit_stock = pd.read_sql("SELECT * FROM visit_stock", conn)
retailers = pd.read_sql("SELECT * FROM retailers", conn)
products = pd.read_sql("SELECT * FROM products", conn)

In [4]:
#Feature Engineering



In [5]:
sales['Date'] = pd.to_datetime(sales['Date'])

In [6]:
cutoff_date = datetime.today() - timedelta(days=90)
recent_sales = sales[sales["Date"] >= cutoff_date]
recent_counts = recent_sales.groupby(["Retailer_ID", "Product_ID"]).size().reset_index(name="Recent_Purchase_Count")
recent_counts.shape

(2984, 3)

In [7]:
retailer_city_map = retailers[["Retailer_ID", "City"]]
sales_with_city = sales.merge(retailer_city_map, on="Retailer_ID", how="left")
city_popularity = sales_with_city.groupby(["City", "Product_ID"]).size().reset_index(name="City_Purchase_Count")


In [8]:
city_popularity.sort_values(by=["City", "City_Purchase_Count"], ascending=[True, False], inplace=True)
city_popularity.head()

,City,Product_ID,City_Purchase_Count
28,Bengaluru,P029,400
12,Bengaluru,P013,392
1,Bengaluru,P002,388
37,Bengaluru,P038,388
32,Bengaluru,P033,385


In [9]:
channel_map = retailers[["Retailer_ID", "Channel"]]
sales_with_channel = sales.merge(channel_map, on="Retailer_ID", how="left")
channel_popularity = sales_with_channel.groupby(["Channel", "Product_ID"]).size().reset_index(name="Channel_Purchase_Count")
channel_popularity.sort_values(by=["Channel", "Channel_Purchase_Count"], ascending=[True, False], inplace=True)
channel_popularity.head()

,Channel,Product_ID,Channel_Purchase_Count
2,General Trade,P003,120
6,General Trade,P007,119
5,General Trade,P006,118
18,General Trade,P019,117
38,General Trade,P039,117


In [10]:
all_retailer_product = pd.merge(retailers[["Retailer_ID", "City", "Channel"]],
                                products[["Product_ID"]],
                                how="cross")

In [11]:
data = all_retailer_product.merge(recent_counts, on=["Retailer_ID", "Product_ID"], how="left")
data = data.merge(city_popularity, on=["City", "Product_ID"], how="left")
data = data.merge(channel_popularity, on=["Channel", "Product_ID"], how="left")
data.fillna(0, inplace=True)

In [12]:
data.shape

(9600, 7)

In [13]:
thresholds = {
    "Recent_Purchase_Count": data["Recent_Purchase_Count"].median(),
    "City_Purchase_Count": data["City_Purchase_Count"].median(),
    "Channel_Purchase_Count": data["Channel_Purchase_Count"].median()
}

thresholds

{'Recent_Purchase_Count': np.float64(0.0),
 'City_Purchase_Count': np.float64(363.0),
 'Channel_Purchase_Count': np.float64(93.0)}

In [14]:
RULES = [
    {
        "column": "Recent_Purchase_Count",
        "condition": lambda x, t=thresholds["Recent_Purchase_Count"]: x > t,
        "score": 3
    },
    {
        "column": "City_Purchase_Count",
        "condition": lambda x, t=thresholds["City_Purchase_Count"]: x > t,
        "score": 2
    },
    {
        "column": "Channel_Purchase_Count",
        "condition": lambda x, t=thresholds["Channel_Purchase_Count"]: x > t,
        "score": 1
    }
]

In [15]:
def dynamic_rule_score(row, rules=RULES):
    score = 0
    for rule in rules:
        value = row.get(rule["column"], 0)
        if rule["condition"](value):
            score += rule["score"]
    return score

In [16]:
data["Rule_Score"] = data.apply(dynamic_rule_score, axis=1)

In [17]:
#Prepare ML Model Input

In [18]:
(data["Recent_Purchase_Count"] > 0).astype(int).value_counts()

Recent_Purchase_Count
0    6616
1    2984
Name: count, dtype: int64

In [19]:
data["Purchased_Label"] = (data["Recent_Purchase_Count"] > 0).astype(int)

In [20]:
# Encode categorical vars
le_city = LabelEncoder()
le_channel = LabelEncoder()
data["City_Enc"] = le_city.fit_transform(data["City"])
data["Channel_Enc"] = le_channel.fit_transform(data["Channel"])

In [21]:
features = ["City_Enc", "Channel_Enc", "City_Purchase_Count", "Channel_Purchase_Count"]
X = data[features]
y = data["Purchased_Label"]

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [23]:
# Predict on full dataset
data["ML_Score"] = clf.predict_proba(X)[:, 1]

In [24]:
data.head()

,Retailer_ID,City,Channel,Product_ID,Recent_Purchase_Count,City_Purchase_Count,Channel_Purchase_Count,Rule_Score,Purchased_Label,City_Enc,Channel_Enc,ML_Score
0,R0001,Bengaluru,Pharmacy,P001,0.0,352,111,1,0,0,3,0.331049
1,R0001,Bengaluru,Pharmacy,P002,0.0,388,120,3,0,0,3,0.288286
2,R0001,Bengaluru,Pharmacy,P003,0.0,382,101,3,0,0,3,0.258182
3,R0001,Bengaluru,Pharmacy,P004,1.0,371,110,6,1,0,3,0.399575
4,R0001,Bengaluru,Pharmacy,P005,0.0,339,96,1,0,0,3,0.334959


In [25]:
#Combine Rule and ML Score
# -------------------------------
data["Final_Score"] = 0.5 * data["ML_Score"] + 0.5 * (data["Rule_Score"] / data["Rule_Score"].max())
data.head()

,Retailer_ID,City,Channel,Product_ID,Recent_Purchase_Count,City_Purchase_Count,Channel_Purchase_Count,Rule_Score,Purchased_Label,City_Enc,Channel_Enc,ML_Score,Final_Score
0,R0001,Bengaluru,Pharmacy,P001,0.0,352,111,1,0,0,3,0.331049,0.248858
1,R0001,Bengaluru,Pharmacy,P002,0.0,388,120,3,0,0,3,0.288286,0.394143
2,R0001,Bengaluru,Pharmacy,P003,0.0,382,101,3,0,0,3,0.258182,0.379091
3,R0001,Bengaluru,Pharmacy,P004,1.0,371,110,6,1,0,3,0.399575,0.699788
4,R0001,Bengaluru,Pharmacy,P005,0.0,339,96,1,0,0,3,0.334959,0.250813


In [26]:
data["Final_Score"].describe()

count    9600.000000
mean        0.351089
std         0.165508
min         0.079701
25%         0.216878
50%         0.340950
75%         0.446842
max         0.742066
Name: Final_Score, dtype: float64

In [27]:
sku_counts = recent_sales.groupby("Retailer_ID")["Product_ID"].nunique().reset_index()
sku_counts.columns = ["Retailer_ID", "Unique_SKUs_Purchased"]

# Merge with data to get top-N per retailer
data = data.merge(sku_counts, on="Retailer_ID", how="left")
data["Unique_SKUs_Purchased"].fillna(5, inplace=True)  # fallback

# Round and clip to limit top-N (max 10)
data["TopN"] = data["Unique_SKUs_Purchased"].apply(lambda x: int(min(round(x * 1.2), 10)))

# Sort and apply dynamic top-N per retailer
data = data.sort_values(["Retailer_ID", "Final_Score"], ascending=[True, False])

recommendations = data.groupby("Retailer_ID").apply(
    lambda x: x.head(int(x["TopN"].iloc[0]))
).reset_index(drop=True)

C:\Users\siman\AppData\Local\Temp\ipykernel_21592\3272556787.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["Unique_SKUs_Purchased"].fillna(5, inplace=True)  # fallback
C:\Users\siman\AppData\Local\Temp\ipykernel_21592\3272556787.py:14: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the groupin

In [28]:
data.head(10)

,Retailer_ID,City,Channel,Product_ID,Recent_Purchase_Count,City_Purchase_Count,Channel_Purchase_Count,Rule_Score,Purchased_Label,City_Enc,Channel_Enc,ML_Score,Final_Score,Unique_SKUs_Purchased,TopN
3,R0001,Bengaluru,Pharmacy,P004,1.0,371,110,6,1,0,3,0.399575,0.699788,8.0,10
19,R0001,Bengaluru,Pharmacy,P020,1.0,383,117,6,1,0,3,0.384836,0.692418,8.0,10
32,R0001,Bengaluru,Pharmacy,P033,1.0,385,97,6,1,0,3,0.383040,0.691520,8.0,10
12,R0001,Bengaluru,Pharmacy,P013,1.0,392,116,6,1,0,3,0.347671,0.673835,8.0,10
13,R0001,Bengaluru,Pharmacy,P014,2.0,375,103,6,1,0,3,0.339934,0.669967,8.0,10
18,R0001,Bengaluru,Pharmacy,P019,1.0,368,99,6,1,0,3,0.251284,0.625642,8.0,10
9,R0001,Bengaluru,Pharmacy,P010,0.0,383,112,3,0,0,3,0.391346,0.445673,8.0,10
16,R0001,Bengaluru,Pharmacy,P017,0.0,369,120,3,0,0,3,0.390048,0.445024,8.0,10
26,R0001,Bengaluru,Pharmacy,P027,1.0,353,109,4,1,0,3,0.212313,0.439490,8.0,10
11,R0001,Bengaluru,Pharmacy,P012,0.0,379,108,3,0,0,3,0.378408,0.439204,8.0,10


In [29]:
recommendations.shape

(2261, 15)

In [30]:
recommendations['Retailer_ID'].nunique(), recommendations['Product_ID'].nunique()

(240, 40)

In [31]:
recommendations.head()

,Retailer_ID,City,Channel,Product_ID,Recent_Purchase_Count,City_Purchase_Count,Channel_Purchase_Count,Rule_Score,Purchased_Label,City_Enc,Channel_Enc,ML_Score,Final_Score,Unique_SKUs_Purchased,TopN
0,R0001,Bengaluru,Pharmacy,P004,1.0,371,110,6,1,0,3,0.399575,0.699788,8.0,10
1,R0001,Bengaluru,Pharmacy,P020,1.0,383,117,6,1,0,3,0.384836,0.692418,8.0,10
2,R0001,Bengaluru,Pharmacy,P033,1.0,385,97,6,1,0,3,0.383040,0.691520,8.0,10
3,R0001,Bengaluru,Pharmacy,P013,1.0,392,116,6,1,0,3,0.347671,0.673835,8.0,10
4,R0001,Bengaluru,Pharmacy,P014,2.0,375,103,6,1,0,3,0.339934,0.669967,8.0,10


In [32]:
# Final Output
final_output = recommendations[["Retailer_ID", "Product_ID", "Final_Score"]].copy()
final_output["Recommendation_Timestamp"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [33]:
# Create table if not exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS product_recommendations_ml (
    Retailer_ID TEXT,
    Product_ID TEXT,
    Final_Score REAL,
    Recommendation_Timestamp TEXT
)
"""
)

# Remove old data
cursor.execute("DELETE FROM product_recommendations_ml")

# Write new recommendations
final_output.to_sql("product_recommendations_ml", conn, if_exists="append", index=False)

print("Product recommendations saved to DB.")
conn.commit()
conn.close()

Product recommendations saved to DB.


In [34]:
final_output.head()

,Retailer_ID,Product_ID,Final_Score,Recommendation_Timestamp
0,R0001,P004,0.699788,2025-07-22 19:25:17
1,R0001,P020,0.692418,2025-07-22 19:25:17
2,R0001,P033,0.691520,2025-07-22 19:25:17
3,R0001,P013,0.673835,2025-07-22 19:25:17
4,R0001,P014,0.669967,2025-07-22 19:25:17


In [35]:
final_output['Product_ID'].value_counts()

Product_ID
P010    149
P033    124
P029    117
P019    100
P004     94
P026     88
P017     87
P015     87
P008     87
P023     83
P013     79
P020     78
P014     77
P002     73
P025     73
P038     70
P003     70
P012     65
P040     57
P037     54
P016     53
P005     53
P035     47
P006     43
P028     40
P018     37
P009     36
P032     34
P001     31
P021     25
P011     24
P030     23
P039     22
P022     18
P007     15
P031     14
P024     12
P027     10
P036      9
P034      3
Name: count, dtype: int64